# Simple CNN notebook

## Import Libraries

In [1]:
%matplotlib inline
import numpy as np
from utils.helper import read_data, blur, to_real, morphology
from tensorflow import keras
FOLDER = "data/data_big/"

## Load data

In [2]:
data_empty = read_data(FOLDER + "empty.csv")
data_1p = read_data(FOLDER + "1p.csv")
data_2p = read_data(FOLDER + "2p.csv")
data_3p = read_data(FOLDER + "3p.csv")
print("----------------Shapes----------------")
print("empty:", data_empty.shape)
print("1p:", data_1p.shape)
print("2p:", data_2p.shape)
print("3p:", data_3p.shape)

----------------Shapes----------------
empty: (480, 5, 3, 64, 64)
1p: (480, 5, 3, 64, 64)
2p: (480, 5, 3, 64, 64)
3p: (480, 5, 3, 64, 64)


In [3]:
LENGTH_EMPTY = data_empty.shape[0]
LENGTH_1P = data_1p.shape[0]
LENGTH_2P = data_2p.shape[0]
LENGTH_3P = data_3p.shape[0]

In [4]:
data_empty_summed = to_real(np.sum(data_empty, axis=1)) 
data_1p_summed = to_real(np.sum(data_1p, axis=1))
data_2p_summed = to_real(np.sum(data_2p, axis=1))
data_3p_summed = to_real(np.sum(data_3p, axis=1))

f:\Projects\hackathon-milan\hackathon-milan\submission\code\utils\helper.py:41: ComplexWarning: Casting complex values to real discards the imaginary part
  return data_real.astype(float)


In [5]:
data_empty_summed[:, :, 32, :] = 0
data_1p_summed[:, :, 32, :] = 0
data_2p_summed[:, :, 32, :] = 0
data_3p_summed[:, :, 32, :] = 0

In [6]:
data_0 = np.moveaxis(data_empty_summed, 1,3)
data_1 = np.moveaxis(data_1p_summed, 1,3)
data_2 = np.moveaxis(data_2p_summed, 1,3)
data_3 = np.moveaxis(data_3p_summed, 1,3)
print(data_0.shape)
print(data_1.shape)
print(data_2.shape)
print(data_3.shape)

(480, 64, 64, 3)
(480, 64, 64, 3)
(480, 64, 64, 3)
(480, 64, 64, 3)


In [7]:
data_tot = np.concatenate((data_0, data_1, data_2, data_3), axis=0)
data_tot.shape

(1920, 64, 64, 3)

### Create Labels

In [8]:
labels_empty = np.zeros(LENGTH_EMPTY)
labels_1p = np.ones(LENGTH_1P)
labels_2p = np.ones(LENGTH_2P) * 2
labels_3p = np.ones(LENGTH_3P) * 3
labels = np.concatenate((labels_empty, labels_1p, labels_2p, labels_3p), axis=0)
print("Labels shape:", labels.shape)

Labels shape: (1920,)


In [9]:
train_labels_one_hot = np.eye(4)[labels.astype(int)]
print("Labels abels one hot shape:", train_labels_one_hot.shape)

Labels abels one hot shape: (1920, 4)


In [10]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data_tot, train_labels_one_hot, test_size=0.2, stratify = train_labels_one_hot)
print(f"Shape train_X: {train_X.shape}")
print(f"Shape test_X: {test_X.shape}")
print(f"Shape train_y: {train_y.shape}")
print(f"Shape test_y: {test_y.shape}")

Shape train_X: (1536, 64, 64, 3)
Shape test_X: (384, 64, 64, 3)
Shape train_y: (1536, 4)
Shape test_y: (384, 4)


In [11]:
models = keras.models
layers = keras.layers

model = models.Sequential(
    [
        layers.Conv2D(8,
                      input_shape = (64,64,3),
                      kernel_size = 5,
                      padding = "same",
                      activation = "relu"),
        layers.MaxPooling2D(2,
                            strides = None,
                            padding = "valid"),
        layers.Conv2D(8,
                      input_shape = (32,32,8),
                      kernel_size = 3,
                      padding = "same",
                      activation = "relu"),
        layers.MaxPooling2D(2,
                    strides = None,
                    padding = "valid"),
        layers.Conv2D(8,
                      input_shape = (16,16,8),
                      kernel_size = 3,
                      padding = "same",
                      activation = "relu"),
        layers.MaxPooling2D(2,
                    strides = None,
                    padding = "valid"),
        layers.Flatten(),
        layers.Dense(4),
        layers.Activation("softmax")
    ]
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 8)         608       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 8)         584       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 8)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 8)         584       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 8)          0

In [13]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)

model.fit(
    train_X, train_y,
    epochs  = 20,
    verbose = 1,
    validation_split = 0.1,
    shuffle= True,
    callbacks=[keras.callbacks.EarlyStopping(patience = 3)]
)

model.save('models_kosta/CNN')

Epoch 1/20
44/44 [==============================] - 2s 44ms/step - loss: 1.0295 - accuracy: 0.5246 - val_loss: 0.8682 - val_accuracy: 0.5844
Epoch 2/20
44/44 [==============================] - 2s 38ms/step - loss: 0.7628 - accuracy: 0.6288 - val_loss: 0.7608 - val_accuracy: 0.6429
Epoch 3/20
44/44 [==============================] - 2s 40ms/step - loss: 0.6430 - accuracy: 0.7113 - val_loss: 0.6632 - val_accuracy: 0.6623
Epoch 4/20
44/44 [==============================] - 2s 38ms/step - loss: 0.5722 - accuracy: 0.7417 - val_loss: 0.5826 - val_accuracy: 0.7273
Epoch 5/20
44/44 [==============================] - 2s 38ms/step - loss: 0.5139 - accuracy: 0.7750 - val_loss: 0.5324 - val_accuracy: 0.7727
Epoch 6/20
44/44 [==============================] - 2s 37ms/step - loss: 0.4615 - accuracy: 0.8025 - val_loss: 0.5795 - val_accuracy: 0.6948
Epoch 7/20
44/44 [==============================] - 2s 38ms/step - loss: 0.4296 - accuracy: 0.8111 - val_loss: 0.4853 - val_accuracy: 0.7922
Epoch 8/20
44

INFO:tensorflow:Assets written to: models_kosta/CNN\assets


INFO:tensorflow:Assets written to: models_kosta/CNN\assets


In [14]:
loss, aacuracy = model.evaluate(test_X, test_y)
print(f"Loss: {loss}")
print(f"Accuracy: {aacuracy}")

12/12 [==============================] - 0s 13ms/step - loss: 0.5104 - accuracy: 0.7578
Loss: 0.5104088187217712
Accuracy: 0.7578125
